In [1]:
import requests, json, itertools
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta

pd.options.display.max_rows =  500

In [2]:
def get_all_stations():
    r = requests.get(
        "https://www.maxjeune-tgvinoui.sncf/api/public/refdata/freeplaces-stations",
        timeout=30,
    )
    stations = {}
    for station in r.json()["stations"]:
        stations[station["station"]] = station["codeStation"]
    return stations

In [3]:
def lookup_one_day(orig: list, dest: list, day: str) -> pd.DataFrame:
    results = []
    for station_tuple in itertools.product(*[orig, dest]):
        r = requests.post(
            "https://www.maxjeune-tgvinoui.sncf/api/public/refdata/search-freeplaces-proposals",
            json={
                "departureDateTime": day,
                "destination": station_tuple[1],
                "origin": station_tuple[0],
            },
            timeout=30,
        )
        try:
            for proposal in r.json()["proposals"]:
                result = {}
                result["start_time"] = datetime.strptime(
                    proposal["departureDate"], "%Y-%m-%dT%H:%M"
                )
                result["date"] = result["start_time"].date()
                result["day"] = result["start_time"].strftime("%A")
                result["hour"] = result["start_time"].hour
                result["number"] = proposal["trainNumber"]
                result["orig"] = proposal["origin"]["label"]
                result["dest"] = proposal["destination"]["label"]
                result["end_time"] = datetime.strptime(
                    proposal["arrivalDate"], "%Y-%m-%dT%H:%M"
                )
                result["seats"] = proposal["freePlaces"]
                result["type"] = proposal["trainEquipment"]
                result["duration"] = round((result["end_time"].timestamp() - result["start_time"].timestamp())/3600, 2)
                result["ts_start"] = result["start_time"].timestamp()*1000
                result["ts_end"] = result["end_time"].timestamp()*1000
                results.append(result)
        except KeyError:
            pass
    return pd.DataFrame.from_dict(results)

In [24]:
        r = requests.post(
            "https://www.maxjeune-tgvinoui.sncf/api/public/refdata/search-freeplaces-proposals",
            json={
                "departureDateTime": "2023-08-29",
                "destination": "FRNTE",
                "origin": "FRDJU",
            },
            timeout=30,
            headers={
                'authority': 'www.maxjeune-tgvinoui.sncf',
                'accept': 'application/json',
                'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
                'content-type': 'application/json',
                'origin': 'https://www.maxjeune-tgvinoui.sncf',
                'referer': 'https://www.maxjeune-tgvinoui.sncf/sncf-connect/max-planner',
                'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"macOS"',
                'sec-fetch-dest': 'empty',
                'sec-fetch-mode': 'cors',
                'sec-fetch-site': 'same-origin',
                'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
                'x-client-app': 'MAX_JEUNE',
                'x-client-app-version': '1.46.1',
            }
        )

In [25]:
r.content

b''

In [4]:
def lookup_date_range_one_way(origs, dests, start, end):
    start = datetime.strptime(start, "%Y-%m-%d").date()
    end = datetime.strptime(end, "%Y-%m-%d").date()

    results = []
    current_date = start
    while current_date <= end:
        results.append(lookup_one_day(origs, dests, current_date.strftime('%Y-%m-%d')))
        current_date += timedelta(days=1)
    
    return pd.concat(results).sort_values(by='start_time')

In [5]:
def lookup_date_range_both_ways(stationsA, stationsB, start, end):
    a_to_b = lookup_date_range_one_way(stationsA, stationsB, start, end)
    a_to_b.insert(0, 'direction', 'outbound')
    b_to_a = lookup_date_range_one_way(stationsB, stationsA, start, end)
    b_to_a.insert(0, 'direction', 'inbound')
    return pd.concat([a_to_b, b_to_a]).sort_values(by='start_time')

In [6]:
result = lookup_date_range_both_ways(["FRDJU"], ["FRNTE"], '2023-08-16', '2023-08-20').reset_index().drop('index', axis=1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
result.head(20)

,direction,start_time,date,day,hour,number,orig,dest,end_time,seats,type,duration,ts_start,ts_end
0,outbound,2023-06-05 07:20:00,2023-06-05,Monday,7,8901,PARIS MONTPARNASSE 1 ET 2,NANTES,2023-06-05 09:20:00,64,INOUI,2.00,1.685942e+12,1.685950e+12
1,outbound,2023-06-05 08:43:00,2023-06-05,Monday,8,5351,MASSY TGV,NANTES,2023-06-05 11:19:00,32,INOUI,2.60,1.685947e+12,1.685957e+12
2,outbound,2023-06-05 10:38:00,2023-06-05,Monday,10,5486,MASSY TGV,NANTES,2023-06-05 13:19:00,5,INOUI,2.68,1.685954e+12,1.685964e+12
3,outbound,2023-06-05 12:20:00,2023-06-05,Monday,12,8807,PARIS MONTPARNASSE 1 ET 2,NANTES,2023-06-05 14:19:00,1,INOUI,1.98,1.685960e+12,1.685968e+12
4,outbound,2023-06-05 12:39:00,2023-06-05,Monday,12,5354,MASSY TGV,NANTES,2023-06-05 14:50:00,1,INOUI,2.18,1.685962e+12,1.685969e+12
5,outbound,2023-06-05 13:32:00,2023-06-05,Monday,13,8875,PARIS MONTPARNASSE 1 ET 2,NANTES,2023-06-05 15:54:00,7,INOUI,2.37,1.685965e+12,1.685973e+12
6,outbound,2023-06-05 14:45:00,2023-06-05,Monday,14,8811,PARIS MONTPARNASSE 1 ET 2,NANTES,2023-06-05 16:50:00,33,INOUI,2.08,1.685969e+12,1.685977e+12
7,outbound,2023-06-05 15:36:00,2023-06-05,Monday,15,8879,PARIS MONTPARNASSE 1 ET 2,NANTES,2023-06-05 17:54:00,50,INOUI,2.30,1.685972e+12,1.685980e+12
8,outbound,2023-06-05 15:45:00,2023-06-05,Monday,15,7623,PARIS MONTPARNASSE 1 ET 2,NANTES,2023-06-05 17:50:00,83,OUIGO,2.08,1.685973e+12,1.685980e+12
9,inbound,2023-06-05 16:05:00,2023-06-05,Monday,16,8880,NANTES,PARIS MONTPARNASSE 1 ET 2,2023-06-05 18:25:00,59,INOUI,2.33,1.685974e+12,1.685982e+12


In [11]:
rg = result.groupby(['date', 'day', 'hour', 'direction', 'orig', 'dest']).agg(total_seats=('seats', 'sum')).reset_index()

In [11]:
rg.head(10)

,date,day,hour,direction,orig,dest,total_seats
0,2023-06-05,Monday,7,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,64
1,2023-06-05,Monday,8,outbound,MASSY TGV,NANTES,32
2,2023-06-05,Monday,10,outbound,MASSY TGV,NANTES,5
3,2023-06-05,Monday,12,outbound,MASSY TGV,NANTES,1
4,2023-06-05,Monday,12,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,1
5,2023-06-05,Monday,13,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,6
6,2023-06-05,Monday,14,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,33
7,2023-06-05,Monday,15,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,134
8,2023-06-05,Monday,16,inbound,NANTES,PARIS MONTPARNASSE 1 ET 2,61
9,2023-06-05,Monday,16,outbound,PARIS MONTPARNASSE 1 ET 2,NANTES,2


In [16]:
for index, row in rg.iterrows():
    if row['direction'] == 'outbound':
        color = "blueviolet"
    else:
        color = "orangered"
    print(f"""
        {{
          day: {(row['date'] - (datetime.strptime('2023-06-04', '%Y-%m-%d')).date()).days+1},
          hour: {row['hour']},
          title: '{row['orig'][0:10]} > {row['dest'][0:10]}',
          alignment: 'left',
          color: '{color}'
        }},
    """)


        {
          day: 1,
          hour: 7,
          title: 'PARIS MONT > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 8,
          title: 'MASSY TGV > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 10,
          title: 'MASSY TGV > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 12,
          title: 'MASSY TGV > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 12,
          title: 'PARIS MONT > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          day: 1,
          hour: 13,
          title: 'PARIS MONT > NANTES',
          alignment: 'left',
          color: 'blueviolet'
        },
    

        {
          

In [23]:
start_dt = datetime.strptime('2023-06-01', '%Y-%m-%d')
end_dt = datetime.strptime('2023-06-10', '%Y-%m-%d')

delta = timedelta(days=1)

# store the dates between two dates in a list
dates = []

while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
    dates.append(start_dt.date().strftime('%A %d/%m'))
    # increment start date by timedelta
    start_dt += delta

In [24]:
dates

['Thursday 01/06',
 'Friday 02/06',
 'Saturday 03/06',
 'Sunday 04/06',
 'Monday 05/06',
 'Tuesday 06/06',
 'Wednesday 07/06',
 'Thursday 08/06',
 'Friday 09/06',
 'Saturday 10/06']